Word'based NER is good, but it has many issues with new words :( Let's try to fix this create new model and create analogue of this model:

![img](https://spacy.io/assets/img/social/preview_default.jpg)

In [ ]:
import sys, json, codecs, csv
import numpy as np
import random
import pandas as pd
import tqdm
import itertools
import seaborn as sns

# visualization
import matplotlib
import numpy as np
from sklearn.model_selection import train_test_split
from itertools import chain
import matplotlib.pyplot as plt
%matplotlib inline  

Import of keras layers

In [ ]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.layers.wrappers import Bidirectional
from keras.models import Model

In [ ]:
from keras.layers import Layer
from keras.layers import concatenate
import keras.backend as K

class MergeDimsLayer(Layer):
    def __init__(self, **kwargs):
        super(MergeDimsLayer, self).__init__(**kwargs)

    def call(self, x, mask=None):
        x_shape = K.shape(x)
        return K.reshape(x, shape=(-1, x_shape[-2], x_shape[-1]))

    def compute_output_shape(self, input_shape):
        return (None, input_shape[-2], input_shape[-1])


class RevertMergeDimsLayer(Layer):
    def __init__(self, **kwargs):
        super(RevertMergeDimsLayer, self).__init__(**kwargs)

    def call(self, inputs, mask=None):
        x = inputs[0]
        word_count = K.shape(inputs[1])[1]
        return K.reshape(x, shape=[-1, word_count, K.shape(x)[-1]])

    def compute_output_shape(self, input_shape):
        x_shape = input_shape[0]
        return (None, None, x_shape[-1])


dataset can be downloaded from this source:
https://github.com/EuropeanaNewspapers/ner-corpora

In [ ]:
with open('../data/ner-corpora/enp_FR.bnf.bio/enp_FR.bnf.bio') as f:
    text = f.read()

In [ ]:
print(text[:100])

In [ ]:
words = []
types = []
for item in text.split('\n'):
    item = item.strip()
    if len(item) == 0:
        continue
    [w, t] = item.split(' ')
    words.append(w)
    types.append(t)

In [ ]:
print(words[:5])
print(types[:5])

In [ ]:
unique_types = list(set(types))
type2id = {x:index for index, x in enumerate(unique_types)}
print(type2id)

In [ ]:
from collections import Counter
word2count = Counter(words)
MAX_WORD_COUNT = 30000
top_words = [x[0] for x in sorted(word2count.items(), key=lambda x: x[1], reverse=True)][:MAX_WORD_COUNT]
word2id = {x:index+1 for index, x in enumerate(top_words)}

In [ ]:
train_dataset, test_dataset = train_test_split(zip(words, types), train_size=0.7)

In [ ]:
MAX_WORD_LEN=10
all_chars = sorted(set(text))
char2id = {x:index+1 for index, x in enumerate(all_chars)}

input_words = Input(shape=(None,), name='input_words')
out_words = Embedding(input_dim=len(word2id)+1, output_dim=200)(input_words)
out_words = Bidirectional(LSTM(200, activation='relu', return_sequences=True))(out_words)

input_chars = Input(shape=(None, MAX_WORD_LEN), name='input_chars')
input_chars_emb = Embedding(input_dim=len(char2id)+1, output_dim=60)(input_chars)
out_chars = MergeDimsLayer()(input_chars_emb)
out_chars = Bidirectional(LSTM(200, activation='elu'))(out_chars)
out_chars  = RevertMergeDimsLayer()([out_chars, input_chars_emb])

result = concatenate([out_words, out_chars])
result = LSTM(100, return_sequences=True)(result)
result = Dense(len(type2id), activation='softmax')(result)
model = Model([input_words, input_chars], result)

model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

In [ ]:
def getCharId(ch):
    return 0 if not ch in char2id else char2id[ch]

def getWordId(w):
    return 0 if not w in word2id else word2id[w]

def gen_batches(dataset, batch_size=64, seq_size=32, batch_count=100):
    # implement it yourself!
        
def encode_text(sentence):
    # implement it yourself

In [ ]:
from keras.callbacks import TensorBoard
import tensorflow as tf
import os, shutil


def write_log(callback, names, logs, batch_no):
    for name, value in zip(names, logs):
        summary = tf.Summary()
        summary_value = summary.value.add()
        summary_value.simple_value = value
        summary_value.tag = name
        callback.writer.add_summary(summary, batch_no)
        callback.writer.flush()
        
logs_dir='./logs2'
if os.path.exists(logs_dir) == True:
    shutil.rmtree(logs_dir)
os.mkdir(logs_dir)
callback = TensorBoard(logs_dir)
callback.set_model(model)

In [ ]:
unique_types

In [ ]:
for epoch in range(100):
    losses = []
    for x,y in gen_batches(train_dataset, batch_count=32):
        loss = model.train_on_batch(x, y)
        losses.append(loss)
    train_loss = np.mean(losses)
        
    losses = []
    for x,y in gen_batches(test_dataset, batch_count=32):
        loss = model.test_on_batch(x, y)
        losses.append(loss)
    test_loss = np.mean(losses)
    
    write_log(callback, ['train', 'test'], [train_loss, test_loss], epoch)

# Test model

In [ ]:
query = test_dataset[160:260]
query_words = [x[0] for x in query]
query_types = [x[1] for x in query]
result = model.predict_on_batch(encode_text(" ".join(query_words)))[0]
for index in range(result.shape[0]):
    w = query_words[index]
    true_type = query_types[index]
    pred_type = unique_types[np.argmax(result[index,:])] 
    print("{}:\t{}\t{}".format(w, pred_type, true_type))

# Home task

- 3 points: Finish and tune the model, compare with previous one. Which one is better? Why?
- 7 points: Compare your model with spacy (https://spacy.io/)